In [1]:
import numpy as np
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
from data_quality import *
import tensorflow as tf
import tensorflow_data_validation as tfdv


import warnings
from tensorflow_metadata.proto.v0 import schema_pb2
warnings.simplefilter('ignore')

In [2]:
clean_flights, dirty_flights = partition_data_files('tmp/FLIGHTS/*.csv')
clean_fb, dirty_fb = partition_data_files('tmp/FBPosts/*.csv')

In [3]:
csv_file = clean_flights[0]
cols = list(pd.read_csv(csv_file, error_bad_lines=False).columns)
data_stats = tfdv.generate_statistics_from_csv(data_location=csv_file, column_names=cols)
tfdv.visualize_statistics(data_stats)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


In [79]:
'''
@description: this is the same function from assignment2; infers schema from a given csv file
@params: csv_file - string csv filepath to read
         column_names - string list, names of the columns along the csv header
@return: schema - inferred schema
'''
def infer_schema_from_csv(csv_file, column_names):
    data_stats = tfdv.generate_statistics_from_csv(data_location=csv_file, column_names=column_names)
    
    #tfdv.visualize_statistics(data_stats)
    schema = tfdv.infer_schema(statistics=data_stats)
    
    return schema 

'''
@description: same function from assignment 2; determines whether the new csv file 
              has anomalies against the schema
@params: csv_file - new csv filepath to compare against schema
         schema - schema against which to compare the new csv file
@return: True - if there are anomalies
          False - if there are no anomalies
'''
def has_anomalies(csv_file, schema):
    #get column names from passed in schema
    cols = [f.name for f in schema.feature].sort()
    
    options = tfdv.StatsOptions(schema=schema, infer_type_from_schema=True)
    data_stats = tfdv.generate_statistics_from_csv(data_location=csv_file,\
                                                   column_names=cols, stats_options=options)
    
    # Check eval data for errors by validating the eval data stats using the previously inferred schema
    anomalies = tfdv.validate_statistics(statistics=data_stats, schema=schema)
    
    tfdv.display_anomalies(anomalies)

    if len(anomalies.anomaly_info) > 0:
        return True
    else:
        return False

'''
@description: takes the current csv file and updates the passed in schema to include its values as "valid"
@params: csv_file - new csv file to add to the schema
         schema - schema to be updated
@return: updated_schema - new schema with anomalies of csv_file param added
'''
def update_schema(csv_file, schema):
    #get column names from passed in schema
    cols = [f.name for f in schema.feature].sort()
    
    options = tfdv.StatsOptions(schema=schema, infer_type_from_schema=True)
    new_batch_stats = tfdv.generate_statistics_from_csv(data_location=csv_file,\
                                                        column_names=cols, stats_options = options)

    # Check eval data for errors by validating the eval data stats using the previously inferred schema
    updated_schema = tfdv.update_schema(schema, new_batch_stats)
    #tfdv.display_schema(schema=updated_schema)

    return updated_schema

'''
@description: updates the schema for num_files number of files
@params: all_filenames - string list of all csv filenames
         num_files - the number of files we want to include in our "acceptable" schema
@return: updated_schema - new schema with anomalies of all num_files number of csv files added
'''
def acceptable_schema(all_filenames, num_files):
    files_to_update = all_filenames[0:num_files]
    successful = True
    
    for i, file in enumerate(files_to_update):
        print("Reading file: ", file)
        
        #for the first file read in, infer its schema
        if i == 0:
            columns = list(pd.read_csv(file, error_bad_lines=False).columns).sort()
            schema = infer_schema_from_csv(file, columns)
        else:
            schema = update_schema(file, schema) #updated schema
            #sanity check to make sure this is working
            if has_anomalies(file, schema):
                print('Unsuccessful schema update on file number {}: {}'.format(i,file))
                successful = False
                break
                
    if successful:
        print("Schema successfully updated for all {} files".format(num_files))
        
    return schema

In [102]:
num_to_update = 5
updated_schema = acceptable_schema(clean_flights,num_to_update)

Reading file:  tmp/FLIGHTS/FLIGHTS_clean_734472.csv
Reading file:  tmp/FLIGHTS/FLIGHTS_clean_734473.csv


Reading file:  tmp/FLIGHTS/FLIGHTS_clean_734474.csv


Reading file:  tmp/FLIGHTS/FLIGHTS_clean_734475.csv


Reading file:  tmp/FLIGHTS/FLIGHTS_clean_734476.csv


Schema successfully updated for all 5 files


Use this updated schema to verify all subsequent clean + dirty files

In [104]:
tfdv.get_feature(updated_schema,'ArrivalGate').distribution_constraints.min_domain_mass = 0.7
updated_anomalies = tfdv.validate_statistics(eval_stats,updated_schema)
tfdv.display_anomalies(updated_anomalies)

,Anomaly short description,Anomaly long description
Feature name,,
'ScheduledDeparture',Unexpected string values,"Examples contain values missing from the schema: 12/07/2011 01:00 PM (<1%), 12/07/2011 01:05 PM (~1%), 12/07/2011 01:30 PM (<1%), 12/07/2011 02:35 PM (~1%), 12/07/2011 02:55 PM (~1%), 12/07/2011 03:05 PM (~1%), 12/07/2011 03:35 PM (<1%), 12/07/2011 04:00 PM (~1%), 12/07/2011 04:15 PM (~1%), 12/07/2011 04:55 PM (<1%), 12/07/2011 06:00 AM (~3%), 12/07/2011 06:30 AM (<1%), 12/07/2011 06:30 PM (~1%), 12/07/2011 06:45 AM (~1%), 12/07/2011 06:45 PM (~1%), 12/07/2011 06:59 AM (~1%), 12/07/2011 07:00 AM (~1%), 12/07/2011 07:10 AM (~4%), 12/07/2011 07:15 AM (~1%), 12/07/2011 07:30 AM (<1%), 12/07/2011 07:45 PM (~3%), 12/07/2011 07:55 AM (~1%), 12/07/2011 08:00 PM (~1%), 12/07/2011 08:20 PM (<1%), 12/07/2011 08:25 PM (<1%), 12/07/2011 08:35 AM (~2%), 12/07/2011 09:00 PM (~1%), 12/07/2011 09:05 AM (~1%), 12/07/2011 10:15 AM (~1%), 12/07/2011 10:40 AM (~3%), 12/07/2011 10:45 AM (<1%), 12/07/2011 11:25 AM (<1%), 12/07/2011 11:25 PM (~1%), 12/07/2011 11:35 AM (<1%), 12/07/2011 11:45 AM (<1%), 12/07/2011 11:50 AM (~1%), 12/07/2011 11:55 PM (~1%), 12/07/2011 12:00 PM (~1%), 12/07/2011 12:10 PM (~1%), 12/07/2011 12:15 PM (<1%), Wed._ Dec.07_ 2011 10:25 a.m. (<1%), Wed._ Dec.07_ 2011 11:25 a.m. (<1%), Wed._ Dec.07_ 2011 11:40 a.m. (~1%), Wed._ Dec.07_ 2011 12:30 a.m. (<1%), Wed._ Dec.07_ 2011 2:30 p.m. (<1%), Wed._ Dec.07_ 2011 2:53 p.m. (<1%), Wed._ Dec.07_ 2011 2:55 p.m. (<1%), Wed._ Dec.07_ 2011 4:00 p.m. (<1%), Wed._ Dec.07_ 2011 5:25 p.m. (<1%), Wed._ Dec.07_ 2011 5:30 p.m. (~1%), Wed._ Dec.07_ 2011 6:00 a.m. (<1%), Wed._ Dec.07_ 2011 7:00 p.m. (<1%), Wed._ Dec.07_ 2011 7:10 p.m. (<1%), Wed._ Dec.07_ 2011 7:15 p.m. (<1%), Wed._ Dec.07_ 2011 7:35 a.m. (<1%), Wed._ Dec.07_ 2011 8:05 a.m. (<1%), Wed._ Dec.07_ 2011 8:40 a.m. (~1%), Wed._ Dec.07_ 2011 9:15 a.m. (<1%), Wed._ Dec.07_ 2011 9:45 a.m. (<1%), Wed_ Dec 7 10:39 AM (<1%), Wed_ Dec 7 11:05 AM (<1%), Wed_ Dec 7 11:08 PM (<1%), Wed_ Dec 7 11:15 AM (<1%), Wed_ Dec 7 11:55 AM (<1%), Wed_ Dec 7 12:00 AM (<1%), Wed_ Dec 7 12:30 PM (~3%), Wed_ Dec 7 12:57 PM (~3%), Wed_ Dec 7 1:33 PM (~1%), Wed_ Dec 7 1:45 PM (<1%), Wed_ Dec 7 1:55 PM (~1%), Wed_ Dec 7 2:30 PM (~1%), Wed_ Dec 7 2:55 PM (~1%), Wed_ Dec 7 3:27 PM (<1%), Wed_ Dec 7 3:50 PM (<1%), Wed_ Dec 7 4:00 PM (<1%), Wed_ Dec 7 4:16 PM (<1%), Wed_ Dec 7 4:25 PM (~1%), Wed_ Dec 7 6:00 AM (<1%), Wed_ Dec 7 7:05 AM (<1%), Wed_ Dec 7 7:35 AM (<1%), Wed_ Dec 7 7:53 AM (<1%), Wed_ Dec 7 8:00 AM (~1%), Wed_ Dec 7 8:22 AM (<1%), Wed_ Dec 7 8:29 AM (<1%), Wed_ Dec 7 8:42 AM (~1%), Wed_ Dec 7 8:55 AM (<1%), Wed_ Dec 7 9:05 AM (<1%)."
'DepartureGate',Unexpected string values,"Examples contain values missing from the schema: 41 (~1%), 48B (~1%), 70A (~1%), 70B (~1%), A14 (~1%), C11 (~4%), C16 (~1%), C19 (~1%), D28 (~1%), D30 (~1%), D32 (~1%), D4 (<1%), D48 (~1%), E2 (<1%), E8 (<1%)."
'for_key',Unexpected string values,"Examples contain values missing from the schema: 12/07/2011 01:00 PM (<1%), 12/07/2011 01:05 PM (~1%), 12/07/2011 01:30 PM (<1%), 12/07/2011 02:35 PM (~1%), 12/07/2011 02:55 PM (~1%), 12/07/2011 03:05 PM (~1%), 12/07/2011 03:35 PM (<1%), 12/07/2011 04:00 PM (~1%), 12/07/2011 04:15 PM (~1%), 12/07/2011 04:55 PM (<1%), 12/07/2011 06:00 AM (~3%), 12/07/2011 06:30 AM (<1%), 12/07/2011 06:30 PM (~1%), 12/07/2011 06:45 AM (~1%), 12/07/2011 06:45 PM (~1%), 12/07/2011 06:59 AM (~1%), 12/07/2011 07:00 AM (~1%), 12/07/2011 07:10 AM (~4%), 12/07/2011 07:15 AM (~1%), 12/07/2011 07:30 AM (<1%), 12/07/2011 07:45 PM (~3%), 12/07/2011 07:55 AM (~1%), 12/07/2011 08:00 PM (~1%), 12/07/2011 08:20 PM (<1%), 12/07/2011 08:25 PM (<1%), 12/07/2011 08:35 AM (~2%), 12/07/2011 09:00 PM (~1%), 12/07/2011 09:05 AM (~1%), 12/07/2011 10:15 AM (~1%), 12/07/2011 10:40 AM (~3%), 12/07/2011 10:45 AM (<1%), 12/07/2011 11:25 AM (<1%), 12/07/2011 11:25 PM (~1%), 12/07/2011 11:35 AM (<1%), 12/07/2011 11:45 AM (<1%), 12/07/2011 11:50 AM (~1%), 12/07/2011 11:55 PM (~